In [1]:
import os,glob
import numpy as np
import cv2
import glob
import pickle
import tensorflow as tf
import argparse
import re
import datetime
from tensorflow.keras.layers import  Input,Conv2D,BatchNormalization,Activation,Subtract,LeakyReLU,Add,Average,Lambda,MaxPool2D,Dropout,UpSampling2D,Concatenate,Multiply,GlobalAveragePooling2D,Dense,ZeroPadding2D,AveragePooling2D
from tensorflow.keras.layers import concatenate,Flatten,ConvLSTM2D,LayerNormalization,GlobalAveragePooling2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import CSVLogger, ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Sequential
import tensorflow.keras.backend as K
from sklearn.svm import LinearSVC
import matplotlib.pyplot as plt
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from skimage.feature import hog,local_binary_pattern
from skimage import data, exposure
from tensorflow.keras.layers import Layer 

In [2]:

!git clone https://github.com/ieee8023/covid-chestxray-dataset.git

normal_dir = "./covid-chestxray-dataset/images"
dir1 = os.path.join(normal_dir,"*.png")
dir = os.path.join(normal_dir,"*.jpg")
dir2 = os.path.join(normal_dir,"*.jpeg")
covid_files = glob.glob(dir)
covid_files2 = glob.glob(dir2)
covid_files1 = glob.glob(dir1)
covid_files.extend(covid_files2)
covid_files.extend(covid_files1)
print("Number of covid-19 files")
print(len(covid_files))

Cloning into 'covid-chestxray-dataset'...
remote: Enumerating objects: 3614, done.
remote: Total 3614 (delta 0), reused 0 (delta 0), pack-reused 3614
Receiving objects: 100% (3614/3614), 632.87 MiB | 40.75 MiB/s, done.
Resolving deltas: 100% (1433/1433), done.
Checking out files: 100% (1173/1173), done.
Number of covid-19 files
924


In [3]:
normal_dir = "../input/chest-xray-pneumonia/chest_xray/train/NORMAL"
dir1 = os.path.join(normal_dir,"*.png")
dir2 = os.path.join(normal_dir,"*.jpeg")
dir = os.path.join(normal_dir,"*.jpg")
normal_files = glob.glob(dir)
normal_1 = glob.glob(dir1)
normal_2 = glob.glob(dir2)
normal_files.extend(normal_1)
normal_files.extend(normal_2)
print("Number of normal files")
print(len(normal_files))

normal_dir = "../input/chest-xray-pneumonia/chest_xray/train/PNEUMONIA"
dir1 = os.path.join(normal_dir,"*.png")
dir2 = os.path.join(normal_dir,"*.jpeg")
dir = os.path.join(normal_dir,"*.jpg")
pneumonia_files = glob.glob(dir)
pneumonia_1 = glob.glob(dir1)
pneumonia_2 = glob.glob(dir2)
pneumonia_files.extend(pneumonia_1)
pneumonia_files.extend(pneumonia_2)

train_dic = {}
for f in covid_files[:739]:
  train_dic[f] = [1,0,0]
for f in normal_files[:1072]:
  train_dic[f] = [0,1,0]
for f in pneumonia_files[:3100]:
  train_dic[f] = [0,0,1]
print("Number of Pneumonia files")
print(len(pneumonia_files))

test_dic = {}
for f in covid_files[739:]:
  test_dic[f] = [1,0,0]
for f in normal_files[1072:]:
  test_dic[f] = [0,1,0]
for f in pneumonia_files[3100:]:
  test_dic[f] = [0,0,1]

import random
l = list(train_dic.items())
random.shuffle(l)
l_test = list(test_dic.items())
random.shuffle(l_test)
print(len(l),len(l_test))


train_features = []
covid_dic_list = {}
data = []
data_112 = []
data_56 = []
data_256 = []
labels = []
for i in range(len(l)):
  file_name,label = l[i]
  img = cv2.imread(file_name)
  try:
    img = cv2.resize(img,(224,224),interpolation = cv2.INTER_CUBIC)
    #img2 = cv2.resize(img,(256,256),interpolation = cv2.INTER_CUBIC)
    #img3 = cv2.resize(img,(112,112),interpolation = cv2.INTER_CUBIC)
    #img4 = cv2.resize(img,(56,56),interpolation = cv2.INTER_CUBIC)
    
    img = img.astype('float32')/255.0
    #img2 = img2.astype('float16')/255.0
    #img3 = img3.astype('float32')/255.0
    #img4 = img4.astype('float16')/255.0
    
    #data_56.append(img4)
    #data_112.append(img3)
    #data_256.append(img)2
    data.append(img)
    labels.append(label)

  except:
    print(i,file_name)
    print("Not possible")  
train_data = np.array(data)
print("Shape of training data")
print(train_data.shape)


#train_data_56 = np.array(data_56)
#print(train_data_56.shape)
#train_data_112 = np.array(data_112)
#print(train_data_112.shape)
#train_data_256 = np.array(data_256)
#print(train_data_256.shape)


train_labels = np.array(labels)
print("Shape of training labels")
print(train_labels.shape)    

print('^_^-training data finished-^_^')


Number of normal files
1341
Number of Pneumonia files
3875
4911 1229
Shape of training data
(4911, 224, 224, 3)
Shape of training labels
(4911, 3)
^_^-training data finished-^_^


In [4]:
mobilenet_model = tf.keras.applications.MobileNetV2(include_top=False,weights='imagenet',classes = 3)

9412608/9406464 [==============================] - 0s 0us/step


In [5]:
mobilenet_model.trainable = False

In [6]:
inputs = tf.keras.Input(shape=(224,224,3))
#x = inputs
x = mobilenet_model(inputs)
input_ = tf.expand_dims(x,axis = 1)
nb_chan = 512
ratio = 16
x3 = ConvLSTM2D(filters=512, kernel_size=(1,1),padding = "same")(input_) 
x3 = BatchNormalization(axis=3, momentum=0.0,epsilon=0.0001)(x3)
x3 = Activation('relu')(x3)

#x_add = Add()([x1,x3])
y = tf.keras.layers.GlobalAveragePooling2D()(x3)
y = tf.keras.layers.Dense(nb_chan // ratio, activation='relu')(y)
y = tf.keras.layers.Dense(nb_chan, activation='sigmoid')(y)
y_3 = tf.keras.layers.Multiply()([x3, y])

ratio = 16
#y = tf.keras.layers.GlobalAveragePooling2D()(y_3)
#y = tf.keras.layers.Dense(nb_chan // ratio, activation='relu')(y)
#y = tf.keras.layers.Dense(nb_chan, activation='sigmoid')(y)
#y3 = tf.keras.layers.Multiply()([y_3, y])

#x_add = Add()([y3,x2])
flat = Flatten()(y_3)

dense_1 = Dense(4096,activation = 'relu')(flat)
dense_2 = Dense(4096,activation = 'relu')(dense_1)
prediction = Dense(3,activation = 'softmax')(dense_2)
mobilenet_pred = Model(inputs = inputs,outputs = prediction)
mobilenet_pred.summary()
mobilenet_pred.compile(optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.0002), loss=tf.keras.losses.CategoricalCrossentropy(from_logits = False) , metrics=['accuracy'])

mobilenet_pred.fit(train_data,train_labels,batch_size = 32,epochs = 60)

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
mobilenetv2_1.00_224 (Functiona (None, None, None, 1 2257984     input_2[0][0]                    
__________________________________________________________________________________________________
tf_op_layer_ExpandDims (TensorF [(None, 1, 7, 7, 128 0           mobilenetv2_1.00_224[0][0]       
__________________________________________________________________________________________________
conv_lst_m2d (ConvLSTM2D)       (None, 7, 7, 512)    3672064     tf_op_layer_ExpandDims[0][0]     
_______________________________________________________________________________________

In [7]:
test_features = []
data = []
labels = []
for i in range(len(l_test)):
  file_name,label = l_test[i]
  img = cv2.imread(file_name)
  try:
    img = cv2.resize(img,(224,224),interpolation = cv2.INTER_CUBIC)
    #fd,hog_img = hog(img,orientations=8,pixels_per_cell=(16,16),cells_per_block=(1,1),visualize=True,multichannel=True)
    #feature = hog_img.astype('float16')/255.0   
    img = img.astype('float32')/255.0
    #test_features.append(feature)
    data.append(img)
    labels.append(label)
  except:
    print(file_name,i)  
test_data = np.array(data)
print("Shape of testing data")
print(test_data.shape)

test_labels = np.array(labels)
print("Shape of testing data")
print(test_labels.shape)    
#data = data.reshape((data.shape[0]*data.shape[1],data.shape[2],data.shape[3],1))
#discard_n = len(data)-len(data)//batch_size*batch_size;
#data = np.delete(data,range(discard_n),axis = 0)
print('^_^-testing data finished-^_^')

Shape of testing data
(1229, 224, 224, 3)
Shape of testing data
(1229, 3)
^_^-testing data finished-^_^


In [8]:
mobilenet_pred.evaluate(test_data,test_labels)

39/39 [==============================] - 2s 45ms/step - loss: 0.7141 - accuracy: 0.9756


[0.7140573263168335, 0.9755899310112]

In [9]:
97.88 normal mobilenet
98.12 mobilenet convlstm se 1 block

SyntaxError: invalid syntax (<ipython-input-9-386f7975c1d5>, line 1)

In [10]:
mobilenet_pred.save("mobilenet.h5")

In [11]:
98.37 normal mobilenet
97.88 normal + convlstm
97.96

SyntaxError: invalid syntax (<ipython-input-11-ea372adbe0de>, line 1)